# Stable Diffusion 3.5 Medium - Brachiaria Generation

Generating scientific documentation images of Brachiaria pasture with positive/negative prompts.


In [ ]:
# Install required packages
!pip install diffusers transformers accelerate torch torchvision huggingface_hub --quiet

In [ ]:
# Import libraries
import torch
from diffusers import StableDiffusion3Pipeline
from huggingface_hub import login
import matplotlib.pyplot as plt
from PIL import Image
import os
from datetime import datetime

In [ ]:
# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Hugging Face login (required for SD 3.5 Medium)
login()

In [ ]:
# Load the Stable Diffusion 3.5 Medium model
print("Loading Stable Diffusion 3.5 Medium...")
pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    torch_dtype=torch.float16,
    device_map="balanced"
)

print("Model loaded successfully!")

In [ ]:
# Define Brachiaria prompts
brachiaria_prompts = {
    'brachiaria_scientific_overhead': {
        'positive': (
            "perfect overhead aerial view of brachiaria brizantha pasture, "
            "thick tropical grass with wide green leaf blades, "
            "dense tufted growth pattern, natural clumping structure, "
            "complete ground coverage, no visible soil, "
            "scientific agricultural documentation, research quality, "
            "natural daylight, uniform soft lighting, "
            "detailed tropical grass texture, realistic vegetation, "
            "brazilian cattle pasture, forage grass field, "
            "top-down perspective, bird's eye documentation"
        ),
        'negative': (
            "ground level, human perspective, horizon visible, "
            "fine thin grass, temperate species, flowers, blooms, "
            "ornamental lawn, decorative setting, shadows, "
            "artistic photography, low quality, distorted view"
        ),
        'description': "Brachiaria científica vista aérea"
    },
    'brachiaria_dense_coverage': {
        'positive': (
            "directly overhead view of dense brachiaria grass field, "
            "thick bright green tropical forage grass, "
            "natural tussock formation, clustered growth pattern, "
            "complete vegetation coverage, lush green pasture, "
            "agricultural field study image, scientific documentation, "
            "natural outdoor lighting, research photography, "
            "wide leaf tropical grass, cattle grazing land, "
            "aerial perspective, top-down agricultural survey"
        ),
        'negative': (
            "side angle, ground perspective, sky background, "
            "narrow leaf grass, cool season grass, clover, legumes, "
            "manicured turf, golf course, dramatic lighting, "
            "poor resolution, blurry, watermarked"
        ),
        'description': "Cobertura densa Brachiaria aérea"
    },
    'brachiaria_natural_pattern': {
        'positive': (
            "overhead aerial documentation of brachiaria pasture, "
            "natural growth pattern of tropical forage grass, "
            "thick green grass blades in irregular clusters, "
            "dense vegetation mat, full ground coverage, "
            "scientific field research image, agricultural study, "
            "soft natural daylight, even illumination, "
            "realistic tropical grass texture, detailed vegetation, "
            "brazilian grassland, livestock pasture, "
            "complete top-down view, research documentation"
        ),
        'negative': (
            "angled view, human eye level, horizon line, "
            "fine grass species, flowering plants, white flowers, "
            "landscaped area, ornamental grass, harsh shadows, "
            "artistic style, low quality, perspective distortion"
        ),
        'description': "Padrão natural Brachiaria vista superior"
    },
    'brachiaria_field_study': {
        'positive': (
            "scientific overhead view of brachiaria grass pasture, "
            "thick tropical grass with broad leaf structure, "
            "natural clumping and tufting growth habit, "
            "dense green vegetation coverage, no bare spots, "
            "agricultural research photography, field documentation, "
            "uniform natural lighting, research quality image, "
            "detailed grass blade texture, realistic pattern, "
            "cattle pasture grass, tropical forage species, "
            "perfect vertical aerial perspective"
        ),
        'negative': (
            "ground level view, perspective angle, trees visible, "
            "thin blade grass, temperate grass, flowers, blooms, "
            "decorative lawn, residential grass, strong shadows, "
            "artistic photography, poor quality, distorted"
        ),
        'description': "Estudo de campo Brachiaria aérea"
    },
    'brachiaria_research_documentation': {
        'positive': (
            "direct overhead aerial view of brachiaria grassland, "
            "thick bright green tropical pasture grass, "
            "natural bunching and clustering growth form, "
            "complete dense ground coverage, healthy vegetation, "
            "scientific agricultural documentation, research image, "
            "natural daylight conditions, soft even lighting, "
            "detailed tropical grass texture, realistic structure, "
            "brazilian cattle pasture, forage grass research, "
            "bird's eye perspective, agricultural field study"
        ),
        'negative': (
            "side perspective, eye level, background elements, "
            "fine textured grass, cool climate species, flowering, "
            "ornamental turf, landscaping, dramatic lighting, "
            "artistic effects, low resolution, blurred image"
        ),
        'description': "Documentação científica Brachiaria"
    }
}

print(f"Loaded {len(brachiaria_prompts)} Brachiaria prompt sets")

In [ ]:
# Create output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"brachiaria_generation_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory: {output_dir}")

In [ ]:
# Function to generate images with positive/negative prompts
def generate_brachiaria_images(prompt_data, num_images=2):
    """
    Generate images using positive and negative prompts
    """
    images = []
    
    positive_prompt = prompt_data['positive']
    negative_prompt = prompt_data['negative']
    
    print(f"Generating {num_images} images for: {prompt_data['description']}")
    print(f"Positive: {positive_prompt[:100]}...")
    print(f"Negative: {negative_prompt[:100]}...")
    
    for i in range(num_images):
        print(f"  Generating image {i+1}/{num_images}...")
        
        with torch.inference_mode():
            image = pipe(
                prompt=positive_prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=25,
                guidance_scale=7.5,
                height=1024,
                width=1024
            ).images[0]
        
        images.append(image)
    
    return images

In [ ]:
# Generate all images
all_generated_images = {}
total_prompts = len(brachiaria_prompts)
current_prompt = 0

for prompt_key, prompt_data in brachiaria_prompts.items():
    current_prompt += 1
    print(f"\n{'='*50}")
    print(f"Processing {current_prompt}/{total_prompts}: {prompt_key}")
    print(f"{'='*50}")
    
    try:
        images = generate_brachiaria_images(prompt_data, num_images=2)
        all_generated_images[prompt_key] = {
            'images': images,
            'description': prompt_data['description']
        }
        print(f"✓ Successfully generated 2 images for {prompt_key}")
        
    except Exception as e:
        print(f"✗ Error generating {prompt_key}: {e}")
        all_generated_images[prompt_key] = {
            'images': [],
            'description': prompt_data['description'],
            'error': str(e)
        }

print(f"\n\nGeneration complete! Generated images for {len([k for k, v in all_generated_images.items() if v['images']])} prompt sets.")

In [ ]:
# Display images in grid layout
def display_image_grid(all_images, cols=2):
    """
    Display all generated images in a grid layout
    """
    prompt_keys = [k for k, v in all_images.items() if v['images']]
    
    if not prompt_keys:
        print("No images to display")
        return
    
    rows = len(prompt_keys)
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, prompt_key in enumerate(prompt_keys):
        images = all_images[prompt_key]['images']
        description = all_images[prompt_key]['description']
        
        for j in range(min(cols, len(images))):
            axes[i, j].imshow(images[j])
            axes[i, j].set_title(f"{description} - Imagem {j+1}", fontsize=10)
            axes[i, j].axis('off')
        
        # Hide unused subplots
        for j in range(len(images), cols):
            axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.savefig(f"{output_dir}/brachiaria_grid_overview.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Grid overview saved as: {output_dir}/brachiaria_grid_overview.png")

# Display the grid
display_image_grid(all_generated_images)

In [ ]:
# Save individual images
def save_individual_images(all_images, output_directory):
    """
    Save each generated image individually
    """
    saved_count = 0
    
    for prompt_key, data in all_images.items():
        if not data['images']:
            continue
            
        for i, image in enumerate(data['images']):
            filename = f"{prompt_key}_image_{i+1}.png"
            filepath = os.path.join(output_directory, filename)
            
            image.save(filepath)
            saved_count += 1
            print(f"Saved: {filename}")
    
    print(f"\nTotal images saved: {saved_count}")
    return saved_count

# Save all images
saved_images = save_individual_images(all_generated_images, output_dir)

In [ ]:
# Generate summary report
print("\n" + "="*60)
print("BRACHIARIA GENERATION SUMMARY")
print("="*60)

successful_prompts = 0
total_images = 0

for prompt_key, data in all_generated_images.items():
    if data['images']:
        successful_prompts += 1
        total_images += len(data['images'])
        print(f"✓ {data['description']}: {len(data['images'])} images")
    else:
        print(f"✗ {data['description']}: Failed - {data.get('error', 'Unknown error')}")

print(f"\nResults:")
print(f"- Successful prompt sets: {successful_prompts}/{len(brachiaria_prompts)}")
print(f"- Total images generated: {total_images}")
print(f"- Output directory: {output_dir}")
print(f"- Grid overview: {output_dir}/brachiaria_grid_overview.png")
print("="*60)